In [10]:
import pandas as pd

from sklearn.ensemble import RandomForestRegressor

from sklearn.model_selection import GridSearchCV

from sklearn.metrics import mean_squared_error, r2_score

import matplotlib.pyplot as plt

In [11]:
F_selection = pd.read_csv("F_Selection/top_N_features_SelectFromModel_RFAll.csv")

# Extract top 10, 20, 30, and 50 features
top_10 = F_selection["Feature"].head(10).tolist()
top_20 = F_selection["Feature"].head(20).tolist()
top_30 = F_selection["Feature"].head(30).tolist()
top_40 = F_selection["Feature"].head(40).tolist()
top_50 = F_selection["Feature"].head(50).tolist()

In [12]:
def Data_Organizer(raw_Data,top):
    
    All_players_dataSet = raw_Data.copy()
    
    All_players_dataSet = All_players_dataSet.drop(columns=All_players_dataSet.columns[All_players_dataSet.columns.str.contains('Unnamed:')])
    All_players_dataSet.reset_index(drop=True, inplace=True)
    
    All_players_dataSet.fillna(0, inplace=True)
    All_players_dataSet.replace('', 0, inplace=True)
    All_players_dataSet.replace('--', 0, inplace=True)
    
    # check if any NaN,empty Strings exists in the dataframe
    any_missing_values = All_players_dataSet.isna().any().any()
    any_empty_values = (All_players_dataSet.applymap(lambda x: x == '')).any().any()
    
    if any_missing_values or any_empty_values:
        print("DataFrame contains missing values or empty strings/spaces.")
    else:
        print("DataFrame does not contain missing values or empty strings/spaces.")
        
    ## seperating our dataFrame
    Y = All_players_dataSet['Points_won']  # Target
    X = All_players_dataSet.drop('Points_won', axis=1)  # Features
    
    X = X.apply(pd.to_numeric, errors='coerce')

    ## keep the specified columns
    columns_to_keep = top
    
    ## keep only wanted columns
    X = X[columns_to_keep]

    return X, Y

In [13]:
all_players_w_add_sorted = pd.read_csv("all_players_w_add_sorted.csv")

C:\Users\Abdou\AppData\Local\Temp\ipykernel_5080\1174582640.py:1: DtypeWarning: Columns (154,155) have mixed types. Specify dtype option on import or set low_memory=False.
  all_players_w_add_sorted = pd.read_csv("all_players_w_add_sorted.csv")


In [14]:
top = top_10

rank_data_10 = pd.DataFrame()

for year in range(1994,2024):
    
    print("Selected Features Top ", top, " Test year:", year)
    
    Test_Data = all_players_w_add_sorted[all_players_w_add_sorted['Year'] == year]
    Train_Data = all_players_w_add_sorted[all_players_w_add_sorted['Year'] < year]
    
    X_train, y_train = Data_Organizer(Train_Data,top)
    X_test , y_test = Data_Organizer(Test_Data,top)
    
    # Initialize and train a RandomForest Regressor model
    rf_model = RandomForestRegressor(n_estimators=200,random_state=42, n_jobs=-1)
    rf_model.fit(X_train, y_train)
    
    # Make predictions on the test set
    y_pred_rf = rf_model.predict(X_test)
    
    # Evaluate the model
    mse_rf = mean_squared_error(y_test, y_pred_rf)
    r2_rf = r2_score(y_test, y_pred_rf)
    
    # Mean Squared Error (MAE)
    print(f'Mean Squared Error (RandomForest Regressor): {mse_rf}')
    print(f'R-squared (RandomForest Regressort): {r2_rf}')
    
    Test_Data_copy = Test_Data.copy()
    Test_Data_copy.loc[:, 'Predicted_points'] = y_pred_rf
    
    # Select only the desired columns
    Test_Data_concatenated = Test_Data_copy[['Player_name','Rank','Pos','Points_won', 'Predicted_points']]
    
    # Display the concatenated DataFrame sorted by 'Points_won'
    display(Test_Data_concatenated.sort_values(by='Predicted_points', ascending=False).head(10))

    Test_Data_concatenated = Test_Data_concatenated.sort_values(by='Predicted_points', ascending=False).head(10)
    
    if rank_data_10.empty:
        new_rank_column = Test_Data_concatenated['Rank'].head(10).rename(f'Rank_{year}')
        rank_data_10 = pd.DataFrame(new_rank_column)
    else:
        new_rank_column = Test_Data_concatenated['Rank'].head(10).rename(f'Rank_{year}')
        rank_data_10 = pd.concat([rank_data_10.reset_index(drop=True), new_rank_column.reset_index(drop=True)], axis=1)

    print("---------------------------------------------------------------------")

display(rank_data_10)

Selected Features Top  ['D-LEBRON_bball', 'DWS_advanced', 'Dpoy_titles', 'predator_defense_raptor', 'DEF WS_nba', 'BLK_nba', 'TOV_100_poss', 'war_reg_season_raptor', 'predator_total_raptor', 'DBPM_advanced']  Test year: 1994
DataFrame does not contain missing values or empty strings/spaces.
DataFrame does not contain missing values or empty strings/spaces.
Mean Squared Error (RandomForest Regressor): 2.5648530968468464
R-squared (RandomForest Regressort): 0.30562343172139483


,Player_name,Rank,Pos,Points_won,Predicted_points
15651,Hakeem Olajuwon,1,C,23.0,41.915
15539,David Robinson,2,C,22.0,13.620
15643,John Stockton,-1.0,PG,0.0,9.475
15741,Tracy Moore,-1.0,SG,0.0,7.880
15739,Patrick Ewing,-1,C,0.0,7.715
15677,Nate McMillan,7T,PG,3.0,6.450
15646,Dell Demps,-1.0,PG,0.0,6.215
15743,Dikembe Mutombo,3,C,19.0,3.085
15477,Scottie Pippen,4,SF,11.0,3.080
15583,Mookie Blaylock,7T,PG,3.0,2.835


---------------------------------------------------------------------
Selected Features Top  ['D-LEBRON_bball', 'DWS_advanced', 'Dpoy_titles', 'predator_defense_raptor', 'DEF WS_nba', 'BLK_nba', 'TOV_100_poss', 'war_reg_season_raptor', 'predator_total_raptor', 'DBPM_advanced']  Test year: 1995
DataFrame does not contain missing values or empty strings/spaces.
DataFrame does not contain missing values or empty strings/spaces.
Mean Squared Error (RandomForest Regressor): 1.6793886046511626
R-squared (RandomForest Regressort): 0.7330717627793228


,Player_name,Rank,Pos,Points_won,Predicted_points
15026,Dikembe Mutombo,1,C,45.0,25.985
14994,Hakeem Olajuwon,3,C,13.0,25.265
15113,David Robinson,4,C,12.0,14.680
15199,Scottie Pippen,2,SF,16.0,13.480
15264,John Stockton,-1.0,PG,0.0,4.820
15051,Patrick Ewing,-1,C,0.0,4.035
15394,Karl Malone,-1.0,PF,0.0,2.175
15018,Tom Hovasse,-1.0,SF,0.0,1.685
15067,Elmer Bennett,-1.0,PG,0.0,1.595
15391,Darrell Armstrong,-1.0,PG,0.0,1.540


---------------------------------------------------------------------
Selected Features Top  ['D-LEBRON_bball', 'DWS_advanced', 'Dpoy_titles', 'predator_defense_raptor', 'DEF WS_nba', 'BLK_nba', 'TOV_100_poss', 'war_reg_season_raptor', 'predator_total_raptor', 'DBPM_advanced']  Test year: 1996
DataFrame does not contain missing values or empty strings/spaces.
DataFrame does not contain missing values or empty strings/spaces.
Mean Squared Error (RandomForest Regressor): 3.329599386503067
R-squared (RandomForest Regressort): 0.5562213099595003


...

,Player_name,Rank,Pos,Points_won,Predicted_points
1022,Rudy Gobert,3.0,C,136.0,191.310
1184,Nikola Jokić,-1.0,C,0.0,87.775
1123,Draymond Green,10,PF,2.0,57.870
628,Mikal Bridges,2.0,SF,202.0,54.295
1111,Giannis Antetokounmpo,6.0,PF,58.0,51.800
1046,Sekou Doumbouya,-1.0,PF,0.0,49.315
992,Paul George,-1.0,PF,0.0,38.095
813,Ricky Rubio,-1.0,PG,0.0,34.735
713,Marcus Smart,1,PG,257.0,28.685
1306,Bam Adebayo,4.0,C,128.0,28.030


---------------------------------------------------------------------
Selected Features Top  ['D-LEBRON_bball', 'DWS_advanced', 'Dpoy_titles', 'predator_defense_raptor', 'DEF WS_nba', 'BLK_nba', 'TOV_100_poss', 'war_reg_season_raptor', 'predator_total_raptor', 'DBPM_advanced']  Test year: 2023
DataFrame does not contain missing values or empty strings/spaces.
DataFrame does not contain missing values or empty strings/spaces.
Mean Squared Error (RandomForest Regressor): 331.27511818555007
R-squared (RandomForest Regressort): 0.2214764786373684


,Player_name,Rank,Pos,Points_won,Predicted_points
176,Giannis Antetokounmpo,6.0,PF,14.0,72.970
59,Jaren Jackson Jr.,1.0,C,391.0,51.410
206,Nic Claxton,9T,C,7.0,36.250
323,Brook Lopez,2.0,C,309.0,33.470
137,Evan Mobley,3.0,PF,101.0,33.235
497,Joel Embiid,9T,C,7.0,15.640
408,Jarrett Allen,-1,C,0.0,11.655
301,Anthony Davis,-1,C,0.0,9.795
436,Draymond Green,4,PF,34.0,6.345
36,Al Horford,-1,C,0.0,4.565


---------------------------------------------------------------------


,Rank_1994,Rank_1995,Rank_1996,Rank_1997,Rank_1998,Rank_1999,Rank_2000,Rank_2001,Rank_2002,Rank_2003,...,Rank_2014,Rank_2015,Rank_2016,Rank_2017,Rank_2018,Rank_2019,Rank_2020,Rank_2021,Rank_2022,Rank_2023
0,1,1,3,2,1,3,1,-1,1,1,...,1.0,1,1,2.0,1.0,1.0,1.0,1.0,3.0,6.0
1,2,3,1,1,2,7T,3T,1,3T,4,...,7.0,4,2,1,-1,2.0,3.0,9.0,-1.0,1.0
2,-1.0,4,5,-1,-1,2,5T,1,-1,3,...,3.0,3.0,3,-1.0,-1.0,5.0,8T,-1,10,9T
3,-1.0,2,4,3,-1.0,1,-1,7,-1,-1,...,5,2,4.0,-1,8.0,3.0,-1.0,-1.0,2.0,2.0
4,-1,-1.0,6,-1.0,3,4,-1,3,-1,-1,...,-1.0,-1.0,10,-1.0,6,6T,11,5.0,6.0,3.0
5,7T,-1,2,-1.0,-1.0,5T,2,-1,-1,-1.0,...,4.0,-1.0,-1.0,3,3,-1.0,2,6.0,-1.0,9T
6,-1.0,-1.0,-1.0,4,-1.0,-1,-1,5T,-1,5.0,...,11,6.0,-1,-1.0,-1.0,-1,10.0,3,-1.0,-1
7,3,-1.0,-1,5,5T,-1,-1.0,-1.0,-1,11,...,2.0,8,7.0,-1.0,-1.0,4,-1.0,-1,-1.0,-1
8,4,-1.0,-1.0,6T,-1,-1.0,-1.0,11T,-1.0,13T,...,-1,-1.0,-1,-1.0,-1.0,-1,-1,-1.0,1,4
9,7T,-1.0,-1.0,-1,5T,-1.0,3T,-1.0,-1,-1.0,...,10.0,5.0,5.0,5T,-1,-1,-1,-1,4.0,-1


In [15]:
top = top_20

rank_data_20 = pd.DataFrame()

for year in range(1994,2024):
    
    print("Selected Features Top ", top, " Test year:", year)
    
    Test_Data = all_players_w_add_sorted[all_players_w_add_sorted['Year'] == year]
    Train_Data = all_players_w_add_sorted[all_players_w_add_sorted['Year'] < year]
    
    X_train, y_train = Data_Organizer(Train_Data,top)
    X_test , y_test = Data_Organizer(Test_Data,top)
    
    # Initialize and train a RandomForest Regressor model
    rf_model = RandomForestRegressor(n_estimators=200,random_state=42, n_jobs=-1)
    rf_model.fit(X_train, y_train)
    
    # Make predictions on the test set
    y_pred_rf = rf_model.predict(X_test)
    
    # Evaluate the model
    mse_rf = mean_squared_error(y_test, y_pred_rf)
    r2_rf = r2_score(y_test, y_pred_rf)
    
    # Mean Squared Error (MAE)
    print(f'Mean Squared Error (RandomForest Regressor): {mse_rf}')
    print(f'R-squared (RandomForest Regressort): {r2_rf}')
    
    Test_Data_copy = Test_Data.copy()
    Test_Data_copy.loc[:, 'Predicted_points'] = y_pred_rf
    
    # Select only the desired columns
    Test_Data_concatenated = Test_Data_copy[['Player_name','Rank','Pos','Points_won', 'Predicted_points']]
    
    # Display the concatenated DataFrame sorted by 'Points_won'
    display(Test_Data_concatenated.sort_values(by='Predicted_points', ascending=False).head(10))

    Test_Data_concatenated = Test_Data_concatenated.sort_values(by='Predicted_points', ascending=False).head(10)
    
    if rank_data_20.empty:
        new_rank_column = Test_Data_concatenated['Rank'].head(10).rename(f'Rank_{year}')
        rank_data_20 = pd.DataFrame(new_rank_column)
    else:
        new_rank_column = Test_Data_concatenated['Rank'].head(10).rename(f'Rank_{year}')
        rank_data_20 = pd.concat([rank_data_20.reset_index(drop=True), new_rank_column.reset_index(drop=True)], axis=1)

    print("---------------------------------------------------------------------")

display(rank_data_20)

Selected Features Top  ['D-LEBRON_bball', 'DWS_advanced', 'Dpoy_titles', 'predator_defense_raptor', 'DEF WS_nba', 'BLK_nba', 'TOV_100_poss', 'war_reg_season_raptor', 'predator_total_raptor', 'DBPM_advanced', 'ORB_opponent', 'Age', 'WINS_espn', 'war_total_raptor', 'Dpoy_votes', 'STL%_nba', 'SOS_misc', 'OPP PTS OFF TOV_nba', 'RPM_espn', 'pace_impact_raptor']  Test year: 1994
DataFrame does not contain missing values or empty strings/spaces.
DataFrame does not contain missing values or empty strings/spaces.
Mean Squared Error (RandomForest Regressor): 2.4748440878378375
R-squared (RandomForest Regressort): 0.3299913562885638


,Player_name,Rank,Pos,Points_won,Predicted_points
15651,Hakeem Olajuwon,1,C,23.0,42.410
15539,David Robinson,2,C,22.0,14.815
15643,John Stockton,-1.0,PG,0.0,9.975
15739,Patrick Ewing,-1,C,0.0,8.655
15677,Nate McMillan,7T,PG,3.0,7.600
15741,Tracy Moore,-1.0,SG,0.0,6.650
15646,Dell Demps,-1.0,PG,0.0,5.475
15477,Scottie Pippen,4,SF,11.0,4.495
15583,Mookie Blaylock,7T,PG,3.0,4.060
15743,Dikembe Mutombo,3,C,19.0,3.240


---------------------------------------------------------------------
Selected Features Top  ['D-LEBRON_bball', 'DWS_advanced', 'Dpoy_titles', 'predator_defense_raptor', 'DEF WS_nba', 'BLK_nba', 'TOV_100_poss', 'war_reg_season_raptor', 'predator_total_raptor', 'DBPM_advanced', 'ORB_opponent', 'Age', 'WINS_espn', 'war_total_raptor', 'Dpoy_votes', 'STL%_nba', 'SOS_misc', 'OPP PTS OFF TOV_nba', 'RPM_espn', 'pace_impact_raptor']  Test year: 1995
DataFrame does not contain missing values or empty strings/spaces.
DataFrame does not contain missing values or empty strings/spaces.
Mean Squared Error (RandomForest Regressor): 1.5630593604651162
R-squared (RandomForest Regressort): 0.7515615631756076


,Player_name,Rank,Pos,Points_won,Predicted_points
15026,Dikembe Mutombo,1,C,45.0,28.835
14994,Hakeem Olajuwon,3,C,13.0,27.735
15113,David Robinson,4,C,12.0,15.885
15199,Scottie Pippen,2,SF,16.0,12.690
15249,Nate McMillan,6T,PG,2.0,5.600
15121,Dennis Rodman,5,PF,11.0,5.410
15151,Gary Payton,6T,PG,2.0,5.395
15264,John Stockton,-1.0,PG,0.0,5.220
15404,Alonzo Mourning,6T,C,2.0,5.215
15051,Patrick Ewing,-1,C,0.0,4.225


---------------------------------------------------------------------
Selected Features Top  ['D-LEBRON_bball', 'DWS_advanced', 'Dpoy_titles', 'predator_defense_raptor', 'DEF WS_nba', 'BLK_nba', 'TOV_100_poss', 'war_reg_season_raptor', 'predator_total_raptor', 'DBPM_advanced', 'ORB_opponent', 'Age', 'WINS_espn', 'war_total_raptor', 'Dpoy_votes', 'STL%_nba', 'SOS_misc', 'OPP PTS OFF TOV_nba', 'RPM_espn', 'pace_impact_raptor']  Test year: 1996
DataFrame does not contain missing values or empty strings/spaces.
DataFrame does not contain missing values or empty strings/spaces.
Mean Squared Error (RandomForest Regressor): 3.3651542433537833
R-squared (RandomForest Regressort): 0.5514824552306856


...

,Player_name,Rank,Pos,Points_won,Predicted_points
1022,Rudy Gobert,3.0,C,136.0,169.030
1184,Nikola Jokić,-1.0,C,0.0,79.530
1306,Bam Adebayo,4.0,C,128.0,67.205
1123,Draymond Green,10,PF,2.0,50.505
992,Paul George,-1.0,PF,0.0,42.765
933,Al Horford,9,C,3.0,41.205
1111,Giannis Antetokounmpo,6.0,PF,58.0,37.150
1046,Sekou Doumbouya,-1.0,PF,0.0,32.400
1007,Joel Embiid,-1,C,0.0,31.300
713,Marcus Smart,1,PG,257.0,22.755


---------------------------------------------------------------------
Selected Features Top  ['D-LEBRON_bball', 'DWS_advanced', 'Dpoy_titles', 'predator_defense_raptor', 'DEF WS_nba', 'BLK_nba', 'TOV_100_poss', 'war_reg_season_raptor', 'predator_total_raptor', 'DBPM_advanced', 'ORB_opponent', 'Age', 'WINS_espn', 'war_total_raptor', 'Dpoy_votes', 'STL%_nba', 'SOS_misc', 'OPP PTS OFF TOV_nba', 'RPM_espn', 'pace_impact_raptor']  Test year: 2023
DataFrame does not contain missing values or empty strings/spaces.
DataFrame does not contain missing values or empty strings/spaces.
Mean Squared Error (RandomForest Regressor): 289.5843636288998
R-squared (RandomForest Regressort): 0.3194531489755523


,Player_name,Rank,Pos,Points_won,Predicted_points
59,Jaren Jackson Jr.,1.0,C,391.0,83.890
176,Giannis Antetokounmpo,6.0,PF,14.0,38.040
137,Evan Mobley,3.0,PF,101.0,37.210
323,Brook Lopez,2.0,C,309.0,35.990
497,Joel Embiid,9T,C,7.0,25.960
301,Anthony Davis,-1,C,0.0,25.775
206,Nic Claxton,9T,C,7.0,25.290
524,Bam Adebayo,5.0,C,18.0,20.550
408,Jarrett Allen,-1,C,0.0,19.045
396,Rudy Gobert,-1.0,C,0.0,11.860


---------------------------------------------------------------------


,Rank_1994,Rank_1995,Rank_1996,Rank_1997,Rank_1998,Rank_1999,Rank_2000,Rank_2001,Rank_2002,Rank_2003,...,Rank_2014,Rank_2015,Rank_2016,Rank_2017,Rank_2018,Rank_2019,Rank_2020,Rank_2021,Rank_2022,Rank_2023
0,1,1,3,2,2,3,1,-1,1,1,...,1.0,1,1,2.0,1.0,1.0,1.0,1.0,3.0,1.0
1,2,3,1,-1,1,1,5T,1,3T,-1,...,7.0,4,2,1,-1,2.0,3.0,9.0,-1.0,6.0
2,-1.0,4,5,1,-1,2,3T,1,-1,4,...,4.0,3.0,3,-1,8.0,5.0,2,-1,4.0,3.0
3,-1,2,4,5,3,7T,-1,-1,-1,3,...,13T,2,4.0,-1.0,6,3.0,11,-1,10,2.0
4,7T,6T,6,4,5T,4,2,7,2,11,...,10.0,6.0,10,3,-1.0,-1,-1.0,6.0,-1.0,9T
5,-1.0,5,2,3,4,5T,-1,3,-1,8,...,5,-1,-1,-1.0,3,-1.0,8T,3,9,-1
6,-1.0,6T,7,6T,11T,-1,-1,11T,3T,-1,...,-1,8,11T,5T,-1,6T,8T,5.0,6.0,9T
7,4,-1.0,8T,-1,11T,7T,7,5T,-1,5.0,...,11,-1.0,7.0,-1,15T,4,-1,11T,-1.0,5.0
8,7T,6T,-1,-1,5T,7T,3T,8T,-1,13T,...,2.0,15T,-1,-1.0,15T,-1,10.0,-1.0,-1,-1
9,3,-1,8T,-1,5T,-1,-1,-1,-1,11,...,13T,10T,-1,-1.0,-1.0,6T,-1,7,1,-1.0


In [16]:
top = top_30

rank_data_30 = pd.DataFrame()

for year in range(1994,2024):
    
    print("Selected Features Top ", top, " Test year:", year)
    
    Test_Data = all_players_w_add_sorted[all_players_w_add_sorted['Year'] == year]
    Train_Data = all_players_w_add_sorted[all_players_w_add_sorted['Year'] < year]
    
    X_train, y_train = Data_Organizer(Train_Data,top)
    X_test , y_test = Data_Organizer(Test_Data,top)
    
    # Initialize and train a RandomForest Regressor model
    rf_model = RandomForestRegressor(n_estimators=200,random_state=42, n_jobs=-1)
    rf_model.fit(X_train, y_train)
    
    # Make predictions on the test set
    y_pred_rf = rf_model.predict(X_test)
    
    # Evaluate the model
    mse_rf = mean_squared_error(y_test, y_pred_rf)
    r2_rf = r2_score(y_test, y_pred_rf)
    
    # Mean Squared Error (MAE)
    print(f'Mean Squared Error (RandomForest Regressor): {mse_rf}')
    print(f'R-squared (RandomForest Regressort): {r2_rf}')
    
    Test_Data_copy = Test_Data.copy()
    Test_Data_copy.loc[:, 'Predicted_points'] = y_pred_rf
    
    # Select only the desired columns
    Test_Data_concatenated = Test_Data_copy[['Player_name','Rank','Pos','Points_won', 'Predicted_points']]
    
    # Display the concatenated DataFrame sorted by 'Points_won'
    display(Test_Data_concatenated.sort_values(by='Predicted_points', ascending=False).head(10))

    Test_Data_concatenated = Test_Data_concatenated.sort_values(by='Predicted_points', ascending=False).head(10)

    if rank_data_30.empty:
        new_rank_column = Test_Data_concatenated['Rank'].head(10).rename(f'Rank_{year}')
        rank_data_30 = pd.DataFrame(new_rank_column)
    else:
        new_rank_column = Test_Data_concatenated['Rank'].head(10).rename(f'Rank_{year}')
        rank_data_30 = pd.concat([rank_data_30.reset_index(drop=True), new_rank_column.reset_index(drop=True)], axis=1)

    print("---------------------------------------------------------------------")

display(rank_data_30)

Selected Features Top  ['D-LEBRON_bball', 'DWS_advanced', 'Dpoy_titles', 'predator_defense_raptor', 'DEF WS_nba', 'BLK_nba', 'TOV_100_poss', 'war_reg_season_raptor', 'predator_total_raptor', 'DBPM_advanced', 'ORB_opponent', 'Age', 'WINS_espn', 'war_total_raptor', 'Dpoy_votes', 'STL%_nba', 'SOS_misc', 'OPP PTS OFF TOV_nba', 'RPM_espn', 'pace_impact_raptor', 'TOV%.1_misc', 'won_dpoy_last_season', 'W_nba', 'DRPM_espn', 'PER_advanced', 'STL_nba', 'DREB_nba', 'DEF RTG_nba', 'OPP PTS PAINT_nba', 'AST_opponent']  Test year: 1994
DataFrame does not contain missing values or empty strings/spaces.
DataFrame does not contain missing values or empty strings/spaces.
Mean Squared Error (RandomForest Regressor): 2.3645685810810813
R-squared (RandomForest Regressort): 0.3598459815070907


,Player_name,Rank,Pos,Points_won,Predicted_points
15651,Hakeem Olajuwon,1,C,23.0,42.755
15539,David Robinson,2,C,22.0,15.315
15643,John Stockton,-1.0,PG,0.0,10.385
15739,Patrick Ewing,-1,C,0.0,7.520
15677,Nate McMillan,7T,PG,3.0,6.055
15741,Tracy Moore,-1.0,SG,0.0,5.645
15477,Scottie Pippen,4,SF,11.0,4.575
15646,Dell Demps,-1.0,PG,0.0,4.015
15583,Mookie Blaylock,7T,PG,3.0,3.760
15743,Dikembe Mutombo,3,C,19.0,3.305


---------------------------------------------------------------------
Selected Features Top  ['D-LEBRON_bball', 'DWS_advanced', 'Dpoy_titles', 'predator_defense_raptor', 'DEF WS_nba', 'BLK_nba', 'TOV_100_poss', 'war_reg_season_raptor', 'predator_total_raptor', 'DBPM_advanced', 'ORB_opponent', 'Age', 'WINS_espn', 'war_total_raptor', 'Dpoy_votes', 'STL%_nba', 'SOS_misc', 'OPP PTS OFF TOV_nba', 'RPM_espn', 'pace_impact_raptor', 'TOV%.1_misc', 'won_dpoy_last_season', 'W_nba', 'DRPM_espn', 'PER_advanced', 'STL_nba', 'DREB_nba', 'DEF RTG_nba', 'OPP PTS PAINT_nba', 'AST_opponent']  Test year: 1995
DataFrame does not contain missing values or empty strings/spaces.
DataFrame does not contain missing values or empty strings/spaces.
Mean Squared Error (RandomForest Regressor): 1.5767333720930232
R-squared (RandomForest Regressort): 0.749388165184539


,Player_name,Rank,Pos,Points_won,Predicted_points
15026,Dikembe Mutombo,1,C,45.0,30.200
14994,Hakeem Olajuwon,3,C,13.0,29.695
15113,David Robinson,4,C,12.0,15.460
15199,Scottie Pippen,2,SF,16.0,10.720
15151,Gary Payton,6T,PG,2.0,5.485
15121,Dennis Rodman,5,PF,11.0,5.340
15404,Alonzo Mourning,6T,C,2.0,5.090
15249,Nate McMillan,6T,PG,2.0,4.805
15264,John Stockton,-1.0,PG,0.0,3.630
15112,Horace Grant,-1,PF,0.0,3.625


---------------------------------------------------------------------
Selected Features Top  ['D-LEBRON_bball', 'DWS_advanced', 'Dpoy_titles', 'predator_defense_raptor', 'DEF WS_nba', 'BLK_nba', 'TOV_100_poss', 'war_reg_season_raptor', 'predator_total_raptor', 'DBPM_advanced', 'ORB_opponent', 'Age', 'WINS_espn', 'war_total_raptor', 'Dpoy_votes', 'STL%_nba', 'SOS_misc', 'OPP PTS OFF TOV_nba', 'RPM_espn', 'pace_impact_raptor', 'TOV%.1_misc', 'won_dpoy_last_season', 'W_nba', 'DRPM_espn', 'PER_advanced', 'STL_nba', 'DREB_nba', 'DEF RTG_nba', 'OPP PTS PAINT_nba', 'AST_opponent']  Test year: 1996
DataFrame does not contain missing values or empty strings/spaces.
DataFrame does not contain missing values or empty strings/spaces.
Mean Squared Error (RandomForest Regressor): 3.201115950920246
R-squared (RandomForest Regressort): 0.5733459559352225


...

,Player_name,Rank,Pos,Points_won,Predicted_points
1022,Rudy Gobert,3.0,C,136.0,195.850
1184,Nikola Jokić,-1.0,C,0.0,74.860
1306,Bam Adebayo,4.0,C,128.0,58.520
1123,Draymond Green,10,PF,2.0,56.380
933,Al Horford,9,C,3.0,52.185
1007,Joel Embiid,-1,C,0.0,46.970
1111,Giannis Antetokounmpo,6.0,PF,58.0,40.990
678,Chris Paul,-1,PG,0.0,38.480
627,Clint Capela,-1.0,C,0.0,36.440
992,Paul George,-1.0,PF,0.0,33.070


---------------------------------------------------------------------
Selected Features Top  ['D-LEBRON_bball', 'DWS_advanced', 'Dpoy_titles', 'predator_defense_raptor', 'DEF WS_nba', 'BLK_nba', 'TOV_100_poss', 'war_reg_season_raptor', 'predator_total_raptor', 'DBPM_advanced', 'ORB_opponent', 'Age', 'WINS_espn', 'war_total_raptor', 'Dpoy_votes', 'STL%_nba', 'SOS_misc', 'OPP PTS OFF TOV_nba', 'RPM_espn', 'pace_impact_raptor', 'TOV%.1_misc', 'won_dpoy_last_season', 'W_nba', 'DRPM_espn', 'PER_advanced', 'STL_nba', 'DREB_nba', 'DEF RTG_nba', 'OPP PTS PAINT_nba', 'AST_opponent']  Test year: 2023
DataFrame does not contain missing values or empty strings/spaces.
DataFrame does not contain missing values or empty strings/spaces.
Mean Squared Error (RandomForest Regressor): 259.26370431034485
R-squared (RandomForest Regressort): 0.39070916902320507


,Player_name,Rank,Pos,Points_won,Predicted_points
59,Jaren Jackson Jr.,1.0,C,391.0,98.945
323,Brook Lopez,2.0,C,309.0,67.185
176,Giannis Antetokounmpo,6.0,PF,14.0,51.930
158,Jrue Holiday,7T,PG,8.0,45.295
137,Evan Mobley,3.0,PF,101.0,43.285
497,Joel Embiid,9T,C,7.0,41.050
524,Bam Adebayo,5.0,C,18.0,35.375
206,Nic Claxton,9T,C,7.0,33.240
301,Anthony Davis,-1,C,0.0,30.805
392,Jacob Gilyard,-1.0,PG,0.0,29.880


---------------------------------------------------------------------


,Rank_1994,Rank_1995,Rank_1996,Rank_1997,Rank_1998,Rank_1999,Rank_2000,Rank_2001,Rank_2002,Rank_2003,...,Rank_2014,Rank_2015,Rank_2016,Rank_2017,Rank_2018,Rank_2019,Rank_2020,Rank_2021,Rank_2022,Rank_2023
0,1,1,3,2,2,3,5T,-1,1,1,...,1.0,1,1,2.0,1.0,1.0,1.0,1.0,3.0,1.0
1,2,3,1,-1,1,1,1,1,3T,-1,...,7.0,4,2,1,-1,2.0,3.0,9.0,-1.0,2.0
2,-1.0,4,5,1,-1,2,3T,1,-1,3,...,4.0,3.0,3,-1,6,5.0,2,5.0,4.0,6.0
3,-1,2,4,5,3,7T,2,-1.0,-1,4,...,13T,2,4.0,-1,8.0,3.0,-1.0,-1,10,7T
4,7T,6T,6,4,5T,7T,-1,3,-1,13T,...,5,6.0,10,-1.0,15T,-1,11,-1,9,3.0
5,-1.0,5,2,3,5T,4,-1,-1,2,-1,...,10.0,10T,7.0,5T,3,-1.0,8T,6.0,-1,9T
6,4,6T,7,6T,-1,-1,-1,7,-1,8,...,13T,8,11T,3,15T,4,10.0,3,6.0,5.0
7,-1.0,6T,-1,-1,4,-1,3T,8T,3T,11,...,2.0,-1,-1,-1.0,-1.0,-1,-1,11T,-1,9T
8,7T,-1.0,8T,-1,5T,5T,7,-1,-1,5.0,...,11,-1.0,-1.0,-1.0,-1.0,-1,8T,7,-1.0,-1
9,3,-1,8T,6T,9T,7T,8T,5T,-1,11,...,3.0,15T,-1.0,-1.0,-1.0,6T,-1,-1.0,-1.0,-1.0


In [17]:
top = top_40

rank_data_40 = pd.DataFrame()

for year in range(1994,2024):
    
    print("Selected Features Top ", top, " Test year:", year)
    
    Test_Data = all_players_w_add_sorted[all_players_w_add_sorted['Year'] == year]
    Train_Data = all_players_w_add_sorted[all_players_w_add_sorted['Year'] < year]
    
    X_train, y_train = Data_Organizer(Train_Data,top)
    X_test , y_test = Data_Organizer(Test_Data,top)
    
    # Initialize and train a RandomForest Regressor model
    rf_model = RandomForestRegressor(n_estimators=200,random_state=42, n_jobs=-1)
    rf_model.fit(X_train, y_train)
    
    # Make predictions on the test set
    y_pred_rf = rf_model.predict(X_test)
    
    # Evaluate the model
    mse_rf = mean_squared_error(y_test, y_pred_rf)
    r2_rf = r2_score(y_test, y_pred_rf)
    
    # Mean Squared Error (MAE)
    print(f'Mean Squared Error (RandomForest Regressor): {mse_rf}')
    print(f'R-squared (RandomForest Regressort): {r2_rf}')
    
    Test_Data_copy = Test_Data.copy()
    Test_Data_copy.loc[:, 'Predicted_points'] = y_pred_rf
    
    # Select only the desired columns
    Test_Data_concatenated = Test_Data_copy[['Player_name','Rank','Pos','Points_won', 'Predicted_points']]
    
    # Display the concatenated DataFrame sorted by 'Points_won'
    display(Test_Data_concatenated.sort_values(by='Predicted_points', ascending=False).head(10))

    Test_Data_concatenated = Test_Data_concatenated.sort_values(by='Predicted_points', ascending=False).head(10)

    if rank_data_40.empty:
        new_rank_column = Test_Data_concatenated['Rank'].head(10).rename(f'Rank_{year}')
        rank_data_40 = pd.DataFrame(new_rank_column)
    else:
        new_rank_column = Test_Data_concatenated['Rank'].head(10).rename(f'Rank_{year}')
        rank_data_40 = pd.concat([rank_data_40.reset_index(drop=True), new_rank_column.reset_index(drop=True)], axis=1)

    print("---------------------------------------------------------------------")

display(rank_data_40)

Selected Features Top  ['D-LEBRON_bball', 'DWS_advanced', 'Dpoy_titles', 'predator_defense_raptor', 'DEF WS_nba', 'BLK_nba', 'TOV_100_poss', 'war_reg_season_raptor', 'predator_total_raptor', 'DBPM_advanced', 'ORB_opponent', 'Age', 'WINS_espn', 'war_total_raptor', 'Dpoy_votes', 'STL%_nba', 'SOS_misc', 'OPP PTS OFF TOV_nba', 'RPM_espn', 'pace_impact_raptor', 'TOV%.1_misc', 'won_dpoy_last_season', 'W_nba', 'DRPM_espn', 'PER_advanced', 'STL_nba', 'DREB_nba', 'DEF RTG_nba', 'OPP PTS PAINT_nba', 'AST_opponent', 'DRB%_misc', 'raptor_total_raptor', 'WS/48_advanced', 'WS_advanced', 'Pace_misc', 'TOV', '2PA_opponent', 'raptor_defense_raptor', 'TRB_team', 'BLK_team']  Test year: 1994
DataFrame does not contain missing values or empty strings/spaces.
DataFrame does not contain missing values or empty strings/spaces.
Mean Squared Error (RandomForest Regressor): 2.2392419481981984
R-squared (RandomForest Regressort): 0.39377536224320897


,Player_name,Rank,Pos,Points_won,Predicted_points
15651,Hakeem Olajuwon,1,C,23.0,37.490
15539,David Robinson,2,C,22.0,16.515
15739,Patrick Ewing,-1,C,0.0,10.425
15741,Tracy Moore,-1.0,SG,0.0,7.790
15643,John Stockton,-1.0,PG,0.0,7.505
15616,Shaquille O'Neal,-1.0,C,0.0,7.045
15646,Dell Demps,-1.0,PG,0.0,7.045
15677,Nate McMillan,7T,PG,3.0,5.020
15477,Scottie Pippen,4,SF,11.0,3.495
15583,Mookie Blaylock,7T,PG,3.0,3.130


---------------------------------------------------------------------
Selected Features Top  ['D-LEBRON_bball', 'DWS_advanced', 'Dpoy_titles', 'predator_defense_raptor', 'DEF WS_nba', 'BLK_nba', 'TOV_100_poss', 'war_reg_season_raptor', 'predator_total_raptor', 'DBPM_advanced', 'ORB_opponent', 'Age', 'WINS_espn', 'war_total_raptor', 'Dpoy_votes', 'STL%_nba', 'SOS_misc', 'OPP PTS OFF TOV_nba', 'RPM_espn', 'pace_impact_raptor', 'TOV%.1_misc', 'won_dpoy_last_season', 'W_nba', 'DRPM_espn', 'PER_advanced', 'STL_nba', 'DREB_nba', 'DEF RTG_nba', 'OPP PTS PAINT_nba', 'AST_opponent', 'DRB%_misc', 'raptor_total_raptor', 'WS/48_advanced', 'WS_advanced', 'Pace_misc', 'TOV', '2PA_opponent', 'raptor_defense_raptor', 'TRB_team', 'BLK_team']  Test year: 1995
DataFrame does not contain missing values or empty strings/spaces.
DataFrame does not contain missing values or empty strings/spaces.
Mean Squared Error (RandomForest Regressor): 1.787883372093023
R-squared (RandomForest Regressort): 0.715827203098

,Player_name,Rank,Pos,Points_won,Predicted_points
15026,Dikembe Mutombo,1,C,45.0,31.355
14994,Hakeem Olajuwon,3,C,13.0,30.825
15113,David Robinson,4,C,12.0,16.205
15199,Scottie Pippen,2,SF,16.0,10.550
15151,Gary Payton,6T,PG,2.0,5.915
15404,Alonzo Mourning,6T,C,2.0,5.860
15077,Mookie Blaylock,-1,PG,0.0,4.915
15121,Dennis Rodman,5,PF,11.0,4.590
15249,Nate McMillan,6T,PG,2.0,4.555
15264,John Stockton,-1.0,PG,0.0,4.455


---------------------------------------------------------------------
Selected Features Top  ['D-LEBRON_bball', 'DWS_advanced', 'Dpoy_titles', 'predator_defense_raptor', 'DEF WS_nba', 'BLK_nba', 'TOV_100_poss', 'war_reg_season_raptor', 'predator_total_raptor', 'DBPM_advanced', 'ORB_opponent', 'Age', 'WINS_espn', 'war_total_raptor', 'Dpoy_votes', 'STL%_nba', 'SOS_misc', 'OPP PTS OFF TOV_nba', 'RPM_espn', 'pace_impact_raptor', 'TOV%.1_misc', 'won_dpoy_last_season', 'W_nba', 'DRPM_espn', 'PER_advanced', 'STL_nba', 'DREB_nba', 'DEF RTG_nba', 'OPP PTS PAINT_nba', 'AST_opponent', 'DRB%_misc', 'raptor_total_raptor', 'WS/48_advanced', 'WS_advanced', 'Pace_misc', 'TOV', '2PA_opponent', 'raptor_defense_raptor', 'TRB_team', 'BLK_team']  Test year: 1996
DataFrame does not contain missing values or empty strings/spaces.
DataFrame does not contain missing values or empty strings/spaces.
Mean Squared Error (RandomForest Regressor): 3.3839425357873214
R-squared (RandomForest Regressort): 0.54897829697

...

,Player_name,Rank,Pos,Points_won,Predicted_points
1022,Rudy Gobert,3.0,C,136.0,207.240
1184,Nikola Jokić,-1.0,C,0.0,74.030
1306,Bam Adebayo,4.0,C,128.0,64.900
1111,Giannis Antetokounmpo,6.0,PF,58.0,51.870
933,Al Horford,9,C,3.0,46.975
1007,Joel Embiid,-1,C,0.0,46.180
1123,Draymond Green,10,PF,2.0,44.580
678,Chris Paul,-1,PG,0.0,41.025
627,Clint Capela,-1.0,C,0.0,36.170
713,Marcus Smart,1,PG,257.0,28.845


---------------------------------------------------------------------
Selected Features Top  ['D-LEBRON_bball', 'DWS_advanced', 'Dpoy_titles', 'predator_defense_raptor', 'DEF WS_nba', 'BLK_nba', 'TOV_100_poss', 'war_reg_season_raptor', 'predator_total_raptor', 'DBPM_advanced', 'ORB_opponent', 'Age', 'WINS_espn', 'war_total_raptor', 'Dpoy_votes', 'STL%_nba', 'SOS_misc', 'OPP PTS OFF TOV_nba', 'RPM_espn', 'pace_impact_raptor', 'TOV%.1_misc', 'won_dpoy_last_season', 'W_nba', 'DRPM_espn', 'PER_advanced', 'STL_nba', 'DREB_nba', 'DEF RTG_nba', 'OPP PTS PAINT_nba', 'AST_opponent', 'DRB%_misc', 'raptor_total_raptor', 'WS/48_advanced', 'WS_advanced', 'Pace_misc', 'TOV', '2PA_opponent', 'raptor_defense_raptor', 'TRB_team', 'BLK_team']  Test year: 2023
DataFrame does not contain missing values or empty strings/spaces.
DataFrame does not contain missing values or empty strings/spaces.
Mean Squared Error (RandomForest Regressor): 256.1653881773399
R-squared (RandomForest Regressort): 0.397990464399

,Player_name,Rank,Pos,Points_won,Predicted_points
59,Jaren Jackson Jr.,1.0,C,391.0,95.545
323,Brook Lopez,2.0,C,309.0,77.795
524,Bam Adebayo,5.0,C,18.0,60.295
176,Giannis Antetokounmpo,6.0,PF,14.0,47.055
158,Jrue Holiday,7T,PG,8.0,41.940
137,Evan Mobley,3.0,PF,101.0,41.080
497,Joel Embiid,9T,C,7.0,36.035
301,Anthony Davis,-1,C,0.0,35.235
206,Nic Claxton,9T,C,7.0,34.935
392,Jacob Gilyard,-1.0,PG,0.0,31.880


---------------------------------------------------------------------


,Rank_1994,Rank_1995,Rank_1996,Rank_1997,Rank_1998,Rank_1999,Rank_2000,Rank_2001,Rank_2002,Rank_2003,...,Rank_2014,Rank_2015,Rank_2016,Rank_2017,Rank_2018,Rank_2019,Rank_2020,Rank_2021,Rank_2022,Rank_2023
0,1,1,3,2,2,3,1,1,1,1,...,1.0,1,1,2.0,1.0,1.0,1.0,1.0,3.0,1.0
1,2,3,1,1,1,1,5T,1,3T,-1,...,7.0,4,2,1,-1,2.0,3.0,-1,-1.0,2.0
2,-1,4,5,-1,-1,2,3T,-1,-1,3,...,13T,2,3,-1,8.0,5.0,2,9.0,4.0,5.0
3,-1.0,2,4,5,3,7T,-1,-1.0,-1,4,...,4.0,3.0,4.0,3,6,3.0,11,5.0,6.0,6.0
4,-1.0,6T,6,4,5T,4,2,-1,-1,13T,...,5,6.0,10,-1.0,15T,6T,-1.0,-1,9,7T
5,-1.0,6T,2,3,5T,-1,-1,3,-1,-1,...,13T,8,-1,5T,3,4,8T,6.0,-1,3.0
6,-1.0,-1,7,6T,4,7T,3T,7,-1,8,...,10.0,10T,7.0,-1.0,-1.0,-1,10.0,3,10,9T
7,7T,5,8T,-1,9T,5T,-1,8T,2,11,...,2.0,-1,11T,-1,-1.0,-1,8T,11T,-1,-1
8,4,6T,8T,-1,-1,-1,7,5T,3T,5.0,...,11,15T,-1.0,-1.0,15T,-1,-1,-1,-1.0,9T
9,7T,-1.0,-1,6T,5T,-1,8T,11T,-1,11,...,8T,-1.0,-1,-1.0,-1.0,-1.0,-1.0,-1.0,1,-1.0


In [18]:
top = top_50

rank_data_50 = pd.DataFrame()

for year in range(1994,2024):
    
    print("Selected Features Top ", top, " Test year:", year)
    
    Test_Data = all_players_w_add_sorted[all_players_w_add_sorted['Year'] == year]
    Train_Data = all_players_w_add_sorted[all_players_w_add_sorted['Year'] < year]
    
    X_train, y_train = Data_Organizer(Train_Data,top)
    X_test , y_test = Data_Organizer(Test_Data,top)
    
    # Initialize and train a RandomForest Regressor model
    rf_model = RandomForestRegressor(n_estimators=200,random_state=42, n_jobs=-1)
    rf_model.fit(X_train, y_train)
    
    # Make predictions on the test set
    y_pred_rf = rf_model.predict(X_test)
    
    # Evaluate the model
    mse_rf = mean_squared_error(y_test, y_pred_rf)
    r2_rf = r2_score(y_test, y_pred_rf)
    
    # Mean Squared Error (MAE)
    print(f'Mean Squared Error (RandomForest Regressor): {mse_rf}')
    print(f'R-squared (RandomForest Regressort): {r2_rf}')
    
    Test_Data_copy = Test_Data.copy()
    Test_Data_copy.loc[:, 'Predicted_points'] = y_pred_rf
    
    # Select only the desired columns
    Test_Data_concatenated = Test_Data_copy[['Player_name','Rank','Pos','Points_won', 'Predicted_points']]
    
    # Display the concatenated DataFrame sorted by 'Points_won'
    display(Test_Data_concatenated.sort_values(by='Predicted_points', ascending=False).head(10))

    Test_Data_concatenated = Test_Data_concatenated.sort_values(by='Predicted_points', ascending=False).head(10)

    if rank_data_50.empty:
        new_rank_column = Test_Data_concatenated['Rank'].head(10).rename(f'Rank_{year}')
        rank_data_50 = pd.DataFrame(new_rank_column)
    else:
        new_rank_column = Test_Data_concatenated['Rank'].head(10).rename(f'Rank_{year}')
        rank_data_50 = pd.concat([rank_data_50.reset_index(drop=True), new_rank_column.reset_index(drop=True)], axis=1)

    print("---------------------------------------------------------------------")

display(rank_data_50)

Selected Features Top  ['D-LEBRON_bball', 'DWS_advanced', 'Dpoy_titles', 'predator_defense_raptor', 'DEF WS_nba', 'BLK_nba', 'TOV_100_poss', 'war_reg_season_raptor', 'predator_total_raptor', 'DBPM_advanced', 'ORB_opponent', 'Age', 'WINS_espn', 'war_total_raptor', 'Dpoy_votes', 'STL%_nba', 'SOS_misc', 'OPP PTS OFF TOV_nba', 'RPM_espn', 'pace_impact_raptor', 'TOV%.1_misc', 'won_dpoy_last_season', 'W_nba', 'DRPM_espn', 'PER_advanced', 'STL_nba', 'DREB_nba', 'DEF RTG_nba', 'OPP PTS PAINT_nba', 'AST_opponent', 'DRB%_misc', 'raptor_total_raptor', 'WS/48_advanced', 'WS_advanced', 'Pace_misc', 'TOV', '2PA_opponent', 'raptor_defense_raptor', 'TRB_team', 'BLK_team', 'DRtg_misc', 'DRB_team', '%BLK_nba', 'BLK%_advanced', 'MOV_misc', 'FGA_opponent', 'AST_team', 'VORP_advanced', '3PA_opponent', 'PF']  Test year: 1994
DataFrame does not contain missing values or empty strings/spaces.
DataFrame does not contain missing values or empty strings/spaces.
Mean Squared Error (RandomForest Regressor): 2.2785

,Player_name,Rank,Pos,Points_won,Predicted_points
15651,Hakeem Olajuwon,1,C,23.0,37.335
15539,David Robinson,2,C,22.0,16.690
15741,Tracy Moore,-1.0,SG,0.0,10.850
15646,Dell Demps,-1.0,PG,0.0,9.540
15643,John Stockton,-1.0,PG,0.0,8.600
15677,Nate McMillan,7T,PG,3.0,8.315
15739,Patrick Ewing,-1,C,0.0,6.520
15616,Shaquille O'Neal,-1.0,C,0.0,5.215
15477,Scottie Pippen,4,SF,11.0,4.000
15743,Dikembe Mutombo,3,C,19.0,3.355


---------------------------------------------------------------------
Selected Features Top  ['D-LEBRON_bball', 'DWS_advanced', 'Dpoy_titles', 'predator_defense_raptor', 'DEF WS_nba', 'BLK_nba', 'TOV_100_poss', 'war_reg_season_raptor', 'predator_total_raptor', 'DBPM_advanced', 'ORB_opponent', 'Age', 'WINS_espn', 'war_total_raptor', 'Dpoy_votes', 'STL%_nba', 'SOS_misc', 'OPP PTS OFF TOV_nba', 'RPM_espn', 'pace_impact_raptor', 'TOV%.1_misc', 'won_dpoy_last_season', 'W_nba', 'DRPM_espn', 'PER_advanced', 'STL_nba', 'DREB_nba', 'DEF RTG_nba', 'OPP PTS PAINT_nba', 'AST_opponent', 'DRB%_misc', 'raptor_total_raptor', 'WS/48_advanced', 'WS_advanced', 'Pace_misc', 'TOV', '2PA_opponent', 'raptor_defense_raptor', 'TRB_team', 'BLK_team', 'DRtg_misc', 'DRB_team', '%BLK_nba', 'BLK%_advanced', 'MOV_misc', 'FGA_opponent', 'AST_team', 'VORP_advanced', '3PA_opponent', 'PF']  Test year: 1995
DataFrame does not contain missing values or empty strings/spaces.
DataFrame does not contain missing values or emp

,Player_name,Rank,Pos,Points_won,Predicted_points
15026,Dikembe Mutombo,1,C,45.0,30.410
14994,Hakeem Olajuwon,3,C,13.0,30.315
15113,David Robinson,4,C,12.0,18.900
15199,Scottie Pippen,2,SF,16.0,10.760
15264,John Stockton,-1.0,PG,0.0,7.920
15404,Alonzo Mourning,6T,C,2.0,6.465
15151,Gary Payton,6T,PG,2.0,5.970
15077,Mookie Blaylock,-1,PG,0.0,5.675
15051,Patrick Ewing,-1,C,0.0,5.560
15249,Nate McMillan,6T,PG,2.0,5.175


---------------------------------------------------------------------
Selected Features Top  ['D-LEBRON_bball', 'DWS_advanced', 'Dpoy_titles', 'predator_defense_raptor', 'DEF WS_nba', 'BLK_nba', 'TOV_100_poss', 'war_reg_season_raptor', 'predator_total_raptor', 'DBPM_advanced', 'ORB_opponent', 'Age', 'WINS_espn', 'war_total_raptor', 'Dpoy_votes', 'STL%_nba', 'SOS_misc', 'OPP PTS OFF TOV_nba', 'RPM_espn', 'pace_impact_raptor', 'TOV%.1_misc', 'won_dpoy_last_season', 'W_nba', 'DRPM_espn', 'PER_advanced', 'STL_nba', 'DREB_nba', 'DEF RTG_nba', 'OPP PTS PAINT_nba', 'AST_opponent', 'DRB%_misc', 'raptor_total_raptor', 'WS/48_advanced', 'WS_advanced', 'Pace_misc', 'TOV', '2PA_opponent', 'raptor_defense_raptor', 'TRB_team', 'BLK_team', 'DRtg_misc', 'DRB_team', '%BLK_nba', 'BLK%_advanced', 'MOV_misc', 'FGA_opponent', 'AST_team', 'VORP_advanced', '3PA_opponent', 'PF']  Test year: 1996
DataFrame does not contain missing values or empty strings/spaces.
DataFrame does not contain missing values or emp

...

,Player_name,Rank,Pos,Points_won,Predicted_points
1022,Rudy Gobert,3.0,C,136.0,200.870
1184,Nikola Jokić,-1.0,C,0.0,80.710
1306,Bam Adebayo,4.0,C,128.0,67.245
933,Al Horford,9,C,3.0,61.225
1123,Draymond Green,10,PF,2.0,52.755
678,Chris Paul,-1,PG,0.0,44.040
1111,Giannis Antetokounmpo,6.0,PF,58.0,42.315
1007,Joel Embiid,-1,C,0.0,36.775
992,Paul George,-1.0,PF,0.0,34.380
627,Clint Capela,-1.0,C,0.0,32.125


---------------------------------------------------------------------
Selected Features Top  ['D-LEBRON_bball', 'DWS_advanced', 'Dpoy_titles', 'predator_defense_raptor', 'DEF WS_nba', 'BLK_nba', 'TOV_100_poss', 'war_reg_season_raptor', 'predator_total_raptor', 'DBPM_advanced', 'ORB_opponent', 'Age', 'WINS_espn', 'war_total_raptor', 'Dpoy_votes', 'STL%_nba', 'SOS_misc', 'OPP PTS OFF TOV_nba', 'RPM_espn', 'pace_impact_raptor', 'TOV%.1_misc', 'won_dpoy_last_season', 'W_nba', 'DRPM_espn', 'PER_advanced', 'STL_nba', 'DREB_nba', 'DEF RTG_nba', 'OPP PTS PAINT_nba', 'AST_opponent', 'DRB%_misc', 'raptor_total_raptor', 'WS/48_advanced', 'WS_advanced', 'Pace_misc', 'TOV', '2PA_opponent', 'raptor_defense_raptor', 'TRB_team', 'BLK_team', 'DRtg_misc', 'DRB_team', '%BLK_nba', 'BLK%_advanced', 'MOV_misc', 'FGA_opponent', 'AST_team', 'VORP_advanced', '3PA_opponent', 'PF']  Test year: 2023
DataFrame does not contain missing values or empty strings/spaces.
DataFrame does not contain missing values or emp

,Player_name,Rank,Pos,Points_won,Predicted_points
59,Jaren Jackson Jr.,1.0,C,391.0,106.460
323,Brook Lopez,2.0,C,309.0,80.955
524,Bam Adebayo,5.0,C,18.0,59.340
176,Giannis Antetokounmpo,6.0,PF,14.0,49.085
137,Evan Mobley,3.0,PF,101.0,43.765
497,Joel Embiid,9T,C,7.0,40.705
158,Jrue Holiday,7T,PG,8.0,39.625
301,Anthony Davis,-1,C,0.0,37.335
206,Nic Claxton,9T,C,7.0,36.940
396,Rudy Gobert,-1.0,C,0.0,31.860


---------------------------------------------------------------------


,Rank_1994,Rank_1995,Rank_1996,Rank_1997,Rank_1998,Rank_1999,Rank_2000,Rank_2001,Rank_2002,Rank_2003,...,Rank_2014,Rank_2015,Rank_2016,Rank_2017,Rank_2018,Rank_2019,Rank_2020,Rank_2021,Rank_2022,Rank_2023
0,1,1,3,2,2,1,1,-1,1,1,...,1.0,1,1,2.0,1.0,1.0,1.0,1.0,3.0,1.0
1,2,3,1,1,1,2,5T,1,3T,-1,...,7.0,4,2,1,-1,2.0,3.0,9.0,-1.0,2.0
2,-1.0,4,5,-1,-1,3,3T,1,-1,3,...,13T,2,3,-1,6,5.0,11,-1,4.0,5.0
3,-1.0,2,4,5,3,7T,-1,-1.0,-1,4,...,4.0,6.0,4.0,3,8.0,3.0,2,5.0,9,6.0
4,-1.0,-1.0,6,4,5T,4,2,-1,-1,13T,...,5,3.0,10,-1.0,3,4,-1.0,-1,10,3.0
5,7T,6T,2,3,5T,-1,-1,7,-1,-1,...,10.0,8,-1,5T,-1.0,6T,10.0,-1.0,-1,9T
6,-1,6T,7,-1,4,7T,3T,3,2,8,...,13T,10T,5.0,-1.0,15T,-1.0,8T,3,6.0,7T
7,-1.0,-1,8T,6T,9T,5T,-1,5T,-1,-1.0,...,2.0,-1,7.0,-1,15T,-1.0,8T,6.0,-1,-1
8,4,-1,8T,-1,-1,-1,7,8T,-1,5.0,...,11,-1,-1,-1.0,-1.0,-1,-1.0,11T,-1.0,9T
9,3,6T,-1,-1,5T,7T,8T,-1,3T,11,...,8T,15T,11T,-1,-1.0,-1,-1,-1.0,-1.0,-1.0


In [19]:
rank_data_10.to_csv('RDF+RDF_All_rank_data_10.csv', index=False)
rank_data_20.to_csv('RDF+RDF_All_rank_data_20.csv', index=False)
rank_data_30.to_csv('RDF+RDF_All_rank_data_30.csv', index=False)
rank_data_40.to_csv('RDF+RDF_All_rank_data_40.csv', index=False)
rank_data_50.to_csv('RDF+RDF_All_rank_data_50.csv', index=False)